# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, we are going to identify the lane mark on the road in a video,anotate the lane mark and output the anotated video. 
---

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
def convert_hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

def convert_hls(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

def mask_whiteyellow(image):
    hls = convert_hls(image)
    # hsv = convert_hsv(image)
    # white color mask
    lower = np.uint8([ 0, 200, 0 ])
    upper = np.uint8([ 255, 255, 255 ])
    white_mask = cv2.inRange(hls, lower, upper)
    # yellow color mask
    lower = np.uint8([ 20, 120, 80 ])
    upper = np.uint8([ 100, 200, 255 ])
    yellow_mask = cv2.inRange(hls, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    maskimage = cv2.bitwise_and(image, image, mask = mask)
    return maskimage

In [ ]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [ ]:
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

In [ ]:
def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=1.):
    return cv2.addWeighted(initial_img, α, img, β, γ)


def line_detect(image):
    # Additional pipeline of masking white and yellow color on image
    whiteyellowimage = mask_whiteyellow(image)
    # First pipeline is to do gray scale
    grayimage = grayscale(whiteyellowimage)
    # Second pipeline is to Gaussion blur and Canny detection
    kernel_size = 5
    gausimage = gaussian_blur(grayimage, kernel_size)
    # Third pipeline is to do Canny detect
    low_threshold = 50
    high_threshold = 150
    cannyimage = canny(gausimage, low_threshold, high_threshold)
    # Foruth pipeline is to select the region of interest or mask image
    imshape = cannyimage.shape
    vertices = np.array([[(0,660),(650, 430), (690, 430), (imshape[1],660)]], dtype=np.int32)
    maskimage = region_of_interest(cannyimage,vertices)
    # Fifth pipeline is to hough space transformation
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 10 #minimum number of pixels making up a line
    max_line_gap = 200    # maximum gap in pixels between connectable line segments
    houghimage, detectlines = hough_lines(maskimage, rho, theta, 
                             threshold, min_line_length, max_line_gap)
    return detectlines

In [ ]:
def slope_xy_avg(detectlines):
    leftlaneslope = []
    rightlaneslope = []
    leftintercept = []
    rightintercept = []
    leftlength = []
    rightlength = []
    count1 = 0
    count2 = 0
    for line in detectlines:
        for x1, y1, x2, y2 in line:
            if x2==x1 or y1==y2:
                continue # ignore grid
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope>0:
                count1 = count1 + 1
                leftlaneslope.append(slope)
                leftintercept.append(intercept)
                leftlength.append(length)
            else:
                count2 = count1 + 1
                rightlaneslope.append(slope)
                rightintercept.append(intercept)
                rightlength.append(length)
    leftslope = np.dot(leftlaneslope, leftlength) / np.sum(leftlength)
    rightslope = np.dot(rightlaneslope, rightlength) / np.sum(rightlength)
    leftintercept = np.dot(leftintercept, leftlength) / np.sum(leftlength)
    rightintercept = np.dot(rightintercept, rightlength) / np.sum(rightlength)
    return leftslope, rightslope, leftintercept, rightintercept
            
global lslope
global rslope
global lintercept
global rintercept
def slope_xy_detect(framebuffer):
    leftslope = 0
    rightslope = 0
    leftintercept = 0
    rightintercept = 0
    count = len(framebuffer)
    for frame in framebuffer:
        lines = line_detect(frame)
        lslope, rslope, lintercept, rintercept = slope_xy_avg(lines)
        leftslope = (leftslope + lslope)
        rightslope = (rightslope + rslope)
        leftintercept = (leftintercept + lintercept)
        rightintercept = (rightintercept + rintercept)
    leftslope = leftslope / count
    rightslope = rightslope / count
    leftintercept = leftintercept / count
    rightintercept = rightintercept / count
    return leftslope, rightslope, leftintercept, rightintercept

In [ ]:
def draw_line(y1, y2, slope, intercept):
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def draw_lane(frame, leftslope, rightslope, leftintercept, rightintercept):
    color = [0, 0, 255] # red lane
    thickness = 20
    lane_image = np.zeros_like(frame)
    y1 = frame.shape[0] 
    y2 = y1*0.6 
    left_lane  = draw_line(y1, y2, leftslope, leftintercept)
    right_lane = draw_line(y1, y2, rightslope, rightintercept)
    cv2.line(lane_image, *left_lane, color, thickness)
    cv2.line(lane_image, *right_lane, color, thickness)
    return lane_image

def lane_detect(framebuffer):    
    # additional pipeline to correct distortion
    # framebuffer = distort_correct(framebuffer)
    # First pipeline is to detect the lane slopes
    lslope, rslope, lintercept, rintercept = slope_xy_detect(framebuffer)
    # print(lslope, rslope, lintercept, rintercept)
    # Second pipeline is to draw the lane image
    laneimage = draw_lane(framebuffer[0], lslope, rslope, lintercept, rintercept)
    return laneimage

In [ ]:
def listshift(l, n):
    return l[n:] + l[:n]

## Test on Videos

We are testing our solution on three provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

`challenge.mp4`


In [ ]:
# Define Variables
font                   = cv2.FONT_HERSHEY_SIMPLEX
UpLeftCornerOfText     = (60,40)
UpRightCornerOfText    = (800,40)
fontScale              = 1
fontColor              = (0,0,255)
lineThick              = 1
lineType               = cv2.LINE_AA
fps                    = 10
framebuffersize        = 10
framebuffer            = []

In [ ]:
# The video processing loop 
while(True):

# Define the OpenCV parameters   
    # read the video source
    cap = cv2.VideoCapture('test_videos/challenge.mp4')
    capfps = cap.get(cv2.CAP_PROP_FPS)  
    # Get current width of frame
    frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
    # Get current height of frame
    frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float
    # Define the output file
    filename = 'result/outvideo.avi'
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    framepersec = fps if fps <= capfps else capfps
    out = cv2.VideoWriter(filename, fourcc,
            int(framepersec), (int(frame_width),int(frame_height)))

    # The frame processing loop    
    while(cap.isOpened()):

        # Firstly buffer a given number of frame for lane identification.       
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            imshape = frame.shape
            # create a frame buffer
            if len(framebuffer) < framebuffersize:
                framebuffer.append(frame)
                continue
            else:
                framebuffer = listshift(framebuffer, -1)
                framebuffer[0] = frame

            # The major lane finding pipeline           
            # detect the lane and draw in a image
            lane = lane_detect(framebuffer)

            # Merge the anotated image to the orignal one            
            # merge the lane image with current frame
            wimage = weighted_img(lane, frame)

            # Using 'q' on keyboard to control the exit of programe  
            cv2.putText(wimage, 'Press \'q\' to pause and quit', UpLeftCornerOfText,
                    font, fontScale, fontColor, lineThick, lineType)   
            # indicate the frame per second   
            if int(fps)<=int(capfps):
                cv2.putText(wimage, str(int(fps))+' FPS', UpRightCornerOfText,
                        font, fontScale, fontColor, lineThick, lineType)
            else:
                cv2.putText(wimage, str(capfps)+' FPS', UpRightCornerOfText,
                        font, fontScale, fontColor, lineThick, lineType)
            # Saves for video
            out.write(wimage)
            # Display the resulting frame
            cv2.imshow('frame',wimage)
            # video play exit
            if cv2.waitKey(int(1000/fps)) & 0xFF == ord('q'):
                break
        else:
            break
    # process exit
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

In [ ]:
# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()